# Second-Order HMM for typos correction

##### Project - TC4

Dhiaeddine YOUSFI & Wafa BOUZOUITA

### Data loading  

In [1]:
#Imports

import pickle
import numpy as np

train_set10 = pickle.load( open( "train10.pkl", "rb" ) )
test_set10 = pickle.load( open( "test10.pkl", "rb" ) )
train_set20 = pickle.load( open( "train20.pkl", "rb" ) )
test_set20 = pickle.load( open( "test20.pkl", "rb" ) )

### HMM parameters estimation 

The Viterbi algorithm is used to find the most probable sequence of states $Z = (z_1,...z_T)$ given a sequence of observations $ X = (x_1,..., x_T)$. To do so, we model the state-observation sequence as being Hidden Markov Models (HMMs) of first and second orders.

In the first order case, the joint probility $p(X,Z)$ can be written as:

$$p(X,Z) = p(z_1)p(x_1 | z_1)\Pi_{k=2}^T p(z_k | z_{k-1})p(x_k | z_k)$$

where $p(z_1)$ is the **initial distribution**, $p(x_k | z_k)$ is the **observation distribution** and $p(z_k | z_{k-1})$ is the **transition distribution**. Those are the three parameters of the HMM and are learned using the training set in the following code section.

For the second order HMM, we just have to estimate a fourth parameter, which represents the **second order transition distribution**, given by $p(z_k | z_{k-1}, z_{k-2})$

In [2]:
def get_HMM_counts(data):
    """
    Builds the observation and state arrays
    @param data: list with (observation, state) chars
    returns:
        state_list: list with unique states
        obs_list: list with unique observations
    """
    state_list = []
    obs_list = []
    
    for seq in data:
        for elem in seq:
            if elem[0] not in obs_list:
                obs_list.append(elem[0])
            if elem[1] not in state_list:
                state_list.append(elem[1])

    return sorted(state_list), sorted(obs_list)   


def initial_distribution(state_list,data):
    """
    Calculates the initial distribution for each state
    @param states: list with unique states
    returns initial_dist: numpy array with the estimated p(z)
    """
    initial_dist = np.zeros(len(state_list))
    for sequence in data:
        for element in sequence:
            state = element[1]
            initial_dist[state_list.index(state)] += 1
    
    initial_dist = initial_dist/np.sum(initial_dist)
    
    return initial_dist 

def observation_distribution(state_list, obs_list, data):
    """
    Calculates the observation distribution for each pair observation|state
    @param data: state,observation pairs
    @param state_list: array of states
    @param observations: array of observations
    returns obs_dist: numpy array with the estimated p(x(k) | z(k))
    """
    obs_dist = np.zeros((len(obs_list),len(state_list)))
    for sequence in data:
        for element in sequence:
            obs = element[0]
            state = element[1]
            obs_dist[obs_list.index(obs),state_list.index(state)] += 1
    for state in range(len(state_list)):
        if np.sum(obs_dist[:,state]) != 0:
            obs_dist[:,state] = obs_dist[:,state]/np.sum(obs_dist[:,state])
    
    return obs_dist
            

def transition_distribution(state_list,data):
    """
    Calculates the transition distribution
    @param data: state,observation pairs
    @param state_list: array of states
    returns trans_dist: numpy array with the estimated p(z(k) | z(k-1))
    """
    trans_dist = np.zeros((len(state_list),len(state_list)))
    for seq in data:
        for idx_elem in range(1,len(seq)):
                trans_dist[state_list.index(seq[idx_elem][1])][state_list.index(seq[idx_elem-1][1])] += 1
    for s0 in range(len(state_list)):
        if np.sum(trans_dist[:,s0]) != 0:
            trans_dist[:,s0] = trans_dist[:,s0]/np.sum(trans_dist[:,s0])
    
    return trans_dist

def second_order_transition(state_list, data):
    """
    Calculates the transition distribution
    @param data: state,observation pairs
    @param state_list: array of states
    returns trans_dist: numpy array with the estimated p(z(k) | z(k-1), z(k-2))
    """
    trans_dist = np.zeros((len(state_list),len(state_list),len(state_list)))
    epsilon = 1e-6
    for seq in data:
        if len(seq) > 2:
            for idx_elem in range(2,len(seq)):
                trans_dist[state_list.index(seq[idx_elem][1])][state_list.index(seq[idx_elem-1][1])][state_list.index(seq[idx_elem-2][1])] += 1
    
    for s0 in range(len(state_list)):
        for s1 in range(len(state_list)):
            total_sum = np.sum(trans_dist[:,s1,s0])
            if total_sum > 0:
                trans_dist[:,s1,s0] = trans_dist[:,s1,s0]/total_sum
            else:
                trans_dist[:,s1,s0] = epsilon
                
    return trans_dist


def get_HMM_params(data):
    """
    Combines all the previous functions and from data,
    calculates all the HMM parameters
    """
    state_list,observation_list = get_HMM_counts(data)
    transition_proba = transition_distribution(state_list, data)
    transition_proba_2_order = second_order_transition(state_list, data)
    observation_proba = observation_distribution(state_list, observation_list, data)
    initial_state_proba = initial_distribution(state_list,data)
    
    return state_list, observation_list, transition_proba, transition_proba_2_order, observation_proba, initial_state_proba

### The HMM class

The HMM class builds a new HMM and contains two main methods:

1) **viterbi**: implementation of the Viterbi algorithm for a first order HMM

2) **viterbi2**: implementation of the same algorithm, but for a second order HMM as proposed by Yang He in the paper:
*"Extended Viterbi algorithm for second order hidden Markov process", [1988 Proceedings] 9th International Conference on Pattern Recognition, Rome, Italy, 1988, pp. 718-720 vol.2.*

In [3]:
class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 transition_proba_2 = None,    
                 observation_proba = None,
                 initial_state_proba = None):
            
            """Builds a new Hidden Markov Model
            state_list is the list of state symbols [q_0...q_(N-1)]
            observation_list is the list of observation symbols [v_0...v_(M-1)]
            transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            
            print ("HMM creating with: ")
            self.N = len(state_list) # The number of states
            self.M = len(observation_list) # The number of words in the vocabulary
            print (str(self.N)+" states")
            print (str(self.M)+" observations")
            self.omega_Y = state_list # Keep the vocabulary of tags
            self.omega_X = observation_list # Keep the vocabulary of tags
            # Init. of the 3 distributions : observation, transition and initial states
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if transition_proba_2 is None:
                self.transition_proba_2 = zeros( (self.N, self.N, self.N), float) 
            else:
                self.transition_proba_2 = transition_proba_2
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            # Since everything will be stored in numpy arrays, it is more convenient and compact to 
            # handle words and tags as indices (integer) for a direct access. However, we also need 
            # to keep the mapping between strings (word or tag) and indices. 
            self.make_indexes()
             

        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities arrays"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
        
        def viterbi(self,seq):
            """
            Viterbi algorithm for first order HMM 
            """
            T = len(seq)
            seq_max = np.zeros(T, dtype = 'int')
            word = []
            mu = np.zeros((self.N,T))
            path_max = np.zeros((self.N,T),dtype = 'int')
            
            # STEP1 - Initialization
            x = seq[0][0]
            for j in self.omega_Y:
                mu[self.Y_index[j],0] = self.initial_state_proba[self.Y_index[j]]*self.observation_proba[self.X_index[x],self.Y_index[j]]
            
            # STEP2 - Recursive Computation
            for i in range(1,T):
                x = seq[i][0]
                for j in self.omega_Y:
                    mu[self.Y_index[j],i] = max([self.observation_proba[self.X_index[x],self.Y_index[j]]*mu[self.Y_index[k],i-1]*self.transition_proba[self.Y_index[j],self.Y_index[k]] for k in self.omega_Y])
                    path_max[self.Y_index[j],i] = np.argmax([mu[self.Y_index[k],i-1]*self.transition_proba[self.Y_index[j],self.Y_index[k]] for k in self.omega_Y])
            
            # STEP3 - Last state
            seq_max[T-1] = np.argmax(mu[:,T-1])
            
            # STEP4 - Backtracing to the first state
            for i in range(T-1,0,-1):
                seq_max[i-1] = path_max[seq_max[i],i]
            
            for s in seq_max:
                word.append(self.omega_Y[s])
                
            return word
        
        def viterbi2(self,seq):
            """
            Viterbi algorithm for second order HMM 
            """
            T = len(seq)
            state_seq = np.zeros(T,dtype = 'int')
            word = []
            
            if T <= 2:
                word = self.viterbi(seq)
            
            else:
                d0 = np.zeros(self.N)
                d = np.zeros((self.N,self.N,T-1))
                c = np.zeros((self.N,self.N,T))
                
                # STEP1 - Initialization
                x0 = seq[0][0]
                x1 = seq[1][0]
                for l in self.omega_Y:
                    d0[self.Y_index[l]] = self.initial_state_proba[self.Y_index[l]]*self.observation_proba[self.X_index[x0],self.Y_index[l]]
                
                for l in self.omega_Y:
                    for m in self.omega_Y:
                        d[self.Y_index[l],self.Y_index[m],0] = d0[self.Y_index[l]]*self.transition_proba[self.Y_index[m],self.Y_index[l]]*self.observation_proba[self.X_index[x1],self.Y_index[m]]
                        
                # STEP2 - Recursive Computation
                for i in range(1,T-1):
                    x = seq[i+1][0]
                    for m in self.omega_Y:
                        for n in self.omega_Y:
                            d[self.Y_index[m],self.Y_index[n],i] = max([d[self.Y_index[l],self.Y_index[m],i-1]*self.transition_proba_2[self.Y_index[n],self.Y_index[m],self.Y_index[l]]*self.observation_proba[self.X_index[x],self.Y_index[n]] for l in self.omega_Y])
                            c[self.Y_index[m],self.Y_index[n],i+1] = np.argmax([d[self.Y_index[l],self.Y_index[m],i-1]*self.transition_proba_2[self.Y_index[n],self.Y_index[m],self.Y_index[l]] for l in self.omega_Y])
                
                # STEP3 - Last two states
                m,n = np.unravel_index(np.argmax(d[:,:,-1]),d[:,:,-1].shape)
                state_seq[T-1] = n
                state_seq[T-2] = m
                
                # STEP4 - Backtracing to the first state
                for i in range(T-3,-1,-1):
                    state_seq[i] = c[state_seq[i+1],state_seq[i+2],i+2]
                    
                for state in state_seq:
                    word.append(self.omega_Y[state])
                
            return word
            

### Performance Analysis

The models are analysed using 4 different metrics:

1) The total **score_char**, which represents the number of correct characters predicted by the model over the total number of characters

2) The total **score_word**, which represents the number of correct word predicted by the model over the total number of words

3) The number of so called **good corrections**, that is: the characters that were wrong in the observations but could be successfully corrected by the model

4) The number of **created errors**, which represent the characters that were originally correct but were incorrectly changed by the algorithm

Additionally, each model is compared with the baseline score of doing nothing.

In [18]:
def classifier_error(hmm, order, data, state_list, observation_list, verbose = True):
    total_score=0
    total_score_word=0
    baseline_score_word = 0
    baseline_score = 0
    good_corrections=0
    created_errors=0
    elements = 0
    errors  = 0
    
    
    print("Running Viterbi algorithm on test set:")
    for k,seq in enumerate(data):
        if order == 1:
            pred_state = hmm.viterbi(seq)
        else:
            pred_state = hmm.viterbi2(seq)
        enc_states = []
        obs_states = []
        
        correct_predicted_letters = 0
        baseline_score_letters = 0
        for i,elem in enumerate(seq):
            
            elements += 1
            enc_state = elem[1]
            enc_obs = elem[0]
            enc_states.append(elem[1])
            obs_states.append(elem[0])
            
            if enc_obs != enc_state : 
                errors +=1
            if enc_obs != enc_state and pred_state[i] == enc_state:
                good_corrections += 1
                total_score += 1
                correct_predicted_letters +=1
            if enc_obs == enc_state:
                baseline_score += 1
                baseline_score_letters +=1
                if pred_state[i] != enc_state:
                    created_errors += 1
                else:
                    total_score += 1  
                    correct_predicted_letters +=1
         
        # to compute the score of words correctly predicted by viterbi
        if correct_predicted_letters == len(seq):
            total_score_word += 1
            
        # to compute the already correct words            
        if baseline_score_letters == len(seq):
            baseline_score_word += 1

        if verbose:
            print("Word: ",k)
            print("Viterbi:  ", pred_state)
            print("Real:     ", enc_states)
            print("Observed: ", obs_states)
            print(" ")
    
    
    
    print("Baseline score (do nothing) = ", (baseline_score/elements)*100, "%")
    print("Baseline score word (do nothing) = ", (baseline_score_word/k)*100, "%")
    print("HMM score = ",total_score/elements*100, "%")
    print("HMM score_word = ",total_score_word/k*100, "%")
    print("Created errors = ",created_errors,"and in %", created_errors*100/errors)
    print("Good corrections = ",good_corrections, " and in % : ", good_corrections*100/ errors )
        

### 1. First order HMM

We start using the first order HMM. To do so, we train and test the model using first the 10% transcription error dataset and, after, the second data set with 20% transcription error.  

#### 10% error dataset

In [19]:
# Use the train set to learn the HMM parameters
state_list10, observation_list10, transition_proba10, transition_proba_2_order10, observation_proba10, initial_state_proba10 = get_HMM_params(train_set10)

# Create a new HMM model using those parameters
model10 = HMM(state_list10, observation_list10, transition_proba10, transition_proba_2_order10, observation_proba10, initial_state_proba10)
print(" ")

# Run the performance analysis
order = 1
classifier_error(model10, order, test_set10, state_list10, observation_list10, verbose = False)

HMM creating with: 
26 states
26 observations
 
Running Viterbi algorithm on test set:
Baseline score (do nothing) =  89.8224043715847 %
Baseline score word (do nothing) =  62.93333333333333 %
HMM score =  93.01912568306011 %
HMM score_word =  74.6 %
Created errors =  65 and in % 8.724832214765101
Good corrections =  299  and in % :  40.13422818791946


#### 20% error dataset

In [20]:
# Use the train set to learn the HMM parameters
state_list20, observation_list20, transition_proba20, transition_proba_2_order20, observation_proba20, initial_state_proba20 = get_HMM_params(train_set20)

# Create a new HMM model using those parameters
model20 = HMM(state_list20, observation_list20, transition_proba20, transition_proba_2_order20, observation_proba20, initial_state_proba20)
print(" ")

# Run the performance analysis
order = 1
classifier_error(model20, order, test_set20, state_list20, observation_list20, verbose = False)

HMM creating with: 
26 states
26 observations
 
Running Viterbi algorithm on test set:
Baseline score (do nothing) =  80.59433227487868 %
Baseline score word (do nothing) =  40.67595612214646 %
HMM score =  86.56761128752022 %
HMM score_word =  57.337681589089826 %
Created errors =  315 and in % 9.725223834516827
Good corrections =  1312  and in % :  40.50632911392405


### 2. Second order HMM

#### 10% error dataset

In [21]:
# Run the performance analysis for second order
order = 2
classifier_error(model10, order, test_set10, state_list10, observation_list10, verbose = False)

Running Viterbi algorithm on test set:
Baseline score (do nothing) =  89.8224043715847 %
Baseline score word (do nothing) =  62.93333333333333 %
HMM score =  95.50546448087431 %
HMM score_word =  83.66666666666667 %
Created errors =  93 and in % 12.483221476510067
Good corrections =  509  and in % :  68.32214765100672


#### 20% error dataset

In [22]:
# Run the performance analysis
order = 2
classifier_error(model20, order, test_set20, state_list20, observation_list20, verbose = False)

Running Viterbi algorithm on test set:
Baseline score (do nothing) =  80.59433227487868 %
Baseline score word (do nothing) =  40.67595612214646 %
HMM score =  91.38457851536756 %
HMM score_word =  71.71657278387192 %
Created errors =  350 and in % 10.805804260574252
Good corrections =  2151  and in % :  66.40938561284347
